<a href="https://colab.research.google.com/github/Kalima83/procesamiento_lenguaje_natural_Desafios/blob/main/desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad. (f1-macro // similaridad de coseno)

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


### 1. Vectorizar los datos de entrenamiento

In [ ]:
# Usamos fit_transform para aprender el vocabulario y el IDF, y luego transformar los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)

# Vectorizar los datos de test
# Usa el transform para aplicar el vocabulario y el IDF aprendidos en el conjunto de entrenamiento
X_test = tfidfvect.transform(newsgroups_test.data)

print("Documentos de entrenamiento vectorizados. Forma de la matriz X_train:")
print(X_train.shape)
print("\nDocumentos de test vectorizados. Forma de la matriz X_test:")
print(X_test.shape)

Documentos de entrenamiento vectorizados. Forma de la matriz X_train:
(11314, 101631)

Documentos de test vectorizados. Forma de la matriz X_test:
(7532, 101631)


####Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Mostrar los 5 documentos más similares de cada uno.

In [ ]:
import random
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Seleccionar 5 índices de documentos al azar del conjunto de entrenamiento
#random_selected_indices = random.sample(range(len(newsgroups_train.data)), 5)

# Se eligen 5 documentos de la primera seleccion random
random_selected_indices = [9685, 2703, 493, 2053, 6285]
print("5 documentos seleccionados al azar:")
print(random_selected_indices)

print("\n================================5 documentos mas similares====================================================")

# Seleccionar los documentos
random_selected_documents = [newsgroups_train.data[i] for i in random_selected_indices]

# Vectorizar los documentos seleccionados al azar utilizando el TfidfVectorizer
X_random_selected = tfidfvect.transform(random_selected_documents)

# Calcular la similaridad coseno entre todos los documentos de entrenamiento
# Tomar los 5 con mayor similaridad
cossim_selected = cosine_similarity(X_random_selected, X_train)


for i, doc_idx in enumerate(random_selected_indices):
    print(f"\n--- Documentos más similares al Documento {doc_idx} ---")
    # Obtener los puntajes de similaridad para el documento seleccionado actual
    current_cossim = cossim_selected[i]

    # Obtener los índices de los documentos ordenados por similaridad de mayor a menor
    most_similar_indices = np.argsort(current_cossim)[::-1][1:6] # [1:6] para obtener los 5 primeros excluyendo el original(coincidencia 1)

    # Imprimir la categoría del documento original
    original_category = newsgroups_train.target_names[newsgroups_train.target[doc_idx]]
    print(f"Categoría del Documento Original: {original_category}")

    # Crear una lista de los datos de los documentos similares
    similar_docs_data = []

    # Recopilar los datos de los 5 documentos más similares
    for similar_doc_idx in most_similar_indices:
        similarity_score = current_cossim[similar_doc_idx]
        similar_doc_category = newsgroups_train.target_names[newsgroups_train.target[similar_doc_idx]]
        similar_docs_data.append({
            "Documento Similar (Índice)": similar_doc_idx,
            "Similaridad Coseno": similarity_score,
            "Categoría del Documento Similar": similar_doc_category
        })

    # Mostrar datos
    df_similar_docs = pd.DataFrame(similar_docs_data)
    display(df_similar_docs)

# Crear el mapa de calor (solo con los valores de similitud)
plt.figure(figsize=(8, 6))
plt.imshow(df_similar_docs[["Similaridad Coseno"]].T, cmap="Blues", aspect="auto")
plt.colorbar(label="Similitud coseno")
plt.xticks(range(len(df_similar_docs)), df_similar_docs["Documento Similar (Índice)"], rotation=45)
plt.yticks([0], ["Documento base"])
plt.title("Mapa de calor de similitudes coseno con los 5 documentos más similares")
plt.xlabel("Índice de documento similar")
plt.ylabel("Documento original")
plt.show()

# --- Calcular la similitud promedio (por si repetís el proceso con varios documentos) ---
avg_sim = df_similar_docs["Similaridad Coseno"].mean()

# Crear el gráfico de dispersión
plt.figure(figsize=(7, 5))
plt.scatter(df_similar_docs["Documento Similar (Índice)"], df_similar_docs["Similaridad Coseno"], s=120)
plt.title("Dispersión de similitudes con documentos similares")
plt.xlabel("Índice de documento similar")
plt.ylabel("Similitud coseno")
plt.grid(True, linestyle="--", alpha=0.6)

# Mostrar valores sobre los puntos
for i, row in df_similar_docs.iterrows():
    plt.text(row["Documento Similar (Índice)"], row["Similaridad Coseno"] + 0.005,
             f"{row['Similaridad Coseno']:.3f}", ha="center", fontsize=9)

plt.show()

print(f"Similitud promedio entre los 5 documentos similares: {avg_sim:.3f}")


5 documentos seleccionados al azar:
[9685, 2703, 493, 2053, 6285]

================================5 documentos mas similares====================================================

--- Documentos más similares al Documento 9685 ---
Categoría del Documento Original: talk.politics.mideast


,Documento Similar (Índice),Similaridad Coseno,Categoría del Documento Similar
0,3687,0.492743,talk.politics.mideast
1,5332,0.351622,talk.politics.mideast
2,2847,0.326602,talk.politics.mideast
3,10637,0.308731,talk.politics.mideast
4,8754,0.285285,talk.religion.misc



--- Documentos más similares al Documento 2703 ---
Categoría del Documento Original: comp.sys.ibm.pc.hardware


,Documento Similar (Índice),Similaridad Coseno,Categoría del Documento Similar
0,3286,0.131933,soc.religion.christian
1,804,0.125007,comp.sys.ibm.pc.hardware
2,4794,0.124221,comp.os.ms-windows.misc
3,5302,0.121863,comp.os.ms-windows.misc
4,4271,0.116019,talk.politics.misc



--- Documentos más similares al Documento 493 ---
Categoría del Documento Original: comp.windows.x


...


--- Documentos más similares al Documento 2053 ---
Categoría del Documento Original: talk.religion.misc


,Documento Similar (Índice),Similaridad Coseno,Categoría del Documento Similar
0,1191,0.220677,talk.politics.misc
1,70,0.209764,talk.politics.mideast
2,3746,0.203593,talk.politics.misc
3,5423,0.197183,talk.politics.mideast
4,6894,0.188944,talk.politics.guns



--- Documentos más similares al Documento 6285 ---
Categoría del Documento Original: rec.autos


,Documento Similar (Índice),Similaridad Coseno,Categoría del Documento Similar
0,8677,0.200890,rec.autos
1,7993,0.183283,rec.autos
2,2749,0.179501,rec.autos
3,6224,0.168774,rec.autos
4,7061,0.166660,rec.motorcycles


NameError: name 'plt' is not defined

### Análisis y Comentarios del Punto 1: Vectorización y Similaridad de Documentos

En este punto, exploramos la vectorización de texto con TF-IDF y calculamos la similaridad coseno entre documentos.

**Observaciones clave:**

*   La similaridad coseno nos permite cuantificar cuán similares son dos documentos en base a su contenido (ponderado por TF-IDF).
*   Al analizar los documentos más similares a uno dado, pudimos ver que a menudo comparten la misma categoría o categorías temáticamente relacionadas, lo que valida la utilidad de la similaridad coseno para encontrar documentos con contenido similar.
*   El análisis de los términos con mayor peso TF-IDF nos ayudó a entender por qué los documentos eran similares, identificando las palabras clave que son más representativas de sus temas.

**Interpretación:**

La vectorización TF-IDF y la similaridad coseno son herramientas efectivas para representar documentos como vectores numéricos y medir su similitud semántica basada en la importancia de sus términos. Esto es fundamental para muchas tareas de procesamiento de lenguaje natural, como la recuperación de información o la clasificación.

### 2. Construir un modelo de clasificación por prototipos (tipo zero-shot)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd

print("===================Clasificación zero-shot basada en similaridad============================")

# Calcular la similaridad coseno entre cada documento de test y todos los documentos de entrenamiento
# X_test: matriz vectorizada de documentos de test
# X_train: matriz vectorizada de documentos de entrenamiento
# El resultado será una matriz donde element (i, j) es la similaridad entre
# el i-ésimo documento de test y el j-th documento de entrenamiento.
# Esto puede tomar un tiempo considerable dependiendo del tamaño de los datasets.
similarity_matrix_test_train = cosine_similarity(X_test, X_train)

print(f"\n1)Matriz de similaridad entre test y train: {similarity_matrix_test_train.shape}")

# Para cada documento de test busca el documento de entrenamiento más similar
most_similar_train_indices = np.argmax(similarity_matrix_test_train, axis=1)

# Las etiquetas predichas para el conjunto de test serán las etiquetas de los documentos de entrenamiento más similares
y_pred_zero_shot = newsgroups_train.target[most_similar_train_indices]


# --- Mostrar Resultados ---

print("\n2) Resultados de la Clasificación Zero-Shot")

# Evaluar el desempeño utilizando F1-score macro
f1_macro_zero_shot = f1_score(y_test, y_pred_zero_shot, average='macro')
print(f"\t F1-score macro: {f1_macro_zero_shot:.4f}")

# Calcular Accuracy
accuracy_zero_shot = accuracy_score(y_test, y_pred_zero_shot)

# Mostrar una muestra de las predicciones en una tabla de pandas
print("\na) Etiquetas Predichas vs. Etiquetas Reales:")
sample_size = 10 # Número de muestras a mostrar

# Crear listas para la tabla
sample_test_indices = list(range(sample_size)) # Tomar los primeros 'sample_size' índices
sample_predicted_categories = [newsgroups_train.target_names[y_pred_zero_shot[i]] for i in sample_test_indices]
sample_actual_categories = [newsgroups_train.target_names[y_test[i]] for i in sample_test_indices]
prediction_matches = ["Sí" if y_pred_zero_shot[i] == y_test[i] else "No" for i in sample_test_indices]

# Crear el DataFrame
df_prediction_sample = pd.DataFrame({
    "Documento Test (Índice)": sample_test_indices,
    "Categoría Predicha": sample_predicted_categories,
    "Categoría Real": sample_actual_categories,
    "Coincide Predicción y Real": prediction_matches
})

# Mostrar el DataFrame
display(df_prediction_sample)

print(f"b) Porcentaje de acierto (Accuracy): {accuracy_zero_shot:.4f}")

### Conclusión 2:

La implementamos un clasificador tipo "zero-shot" basado en la similaridad coseno nos da un resultado similar y aceptable: Porcentaje de acierto (Accuracy): 0.5089 y F1-score macro: 0.5050. Teniendo en cuenta que este consume menos recuros que un clasificador entrenado.


### 3. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test.

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np

print("===================Evaluando MultinomialNB con variacion del parámetros Alpha =======================")


In [ ]:
# Valores de alpha
alpha_values = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 5.0, 10.0]

# Listas para almacenar los resultados
f1_scores = []

print("Experimentando con diferentes valores de alpha para MultinomialNB:")

for alpha in alpha_values:
    mnb_clf = MultinomialNB(alpha=alpha)

    # Entrenar el modelo
    mnb_clf.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = mnb_clf.predict(X_test)

    # Calcular F1-score macro
    f1 = f1_score(y_test, y_pred, average='macro')
    f1_scores.append(f1)

# Mostrar los resultados
print("\nResultados F1-score macro para diferentes valores de alpha:")
for alpha, f1 in zip(alpha_values, f1_scores):
    print(f"Alpha = {alpha:.4f}, F1-score macro = {f1:.4f}")

# Graficar los resultados
plt.figure(figsize=(10, 6))
plt.plot(alpha_values, f1_scores, marker='o')
plt.xscale('log')  # Escala logarítmica para el eje por la variación de alpha
plt.xlabel("Valor de Alpha")
plt.ylabel("F1-score Macro")
plt.title("F1-score Macro vs. Valor de Alpha para MultinomialNB")
plt.grid(True, which="both", ls="--")
plt.show()

In [ ]:
# Valores de alpha
alpha_values = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 5.0, 10.0]

# Listas para almacenar los resultados
f1_scores_cnb = []

print("Experimentando con diferentes valores de alpha para ComplementNB:")

for alpha in alpha_values:
    cnb_clf = ComplementNB(alpha=alpha)

    # Entrenar el modelo
    cnb_clf.fit(X_train, y_train)

    # Hacer predicciones
    y_pred_cnb = cnb_clf.predict(X_test)

    # Calcular F1-score macro
    f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')
    f1_scores_cnb.append(f1_cnb)

# Mostrar los resultados
print("\nResultados F1-score macro para diferentes valores de alpha (ComplementNB):")
for alpha, f1_cnb in zip(alpha_values, f1_scores_cnb):
    print(f"Alpha = {alpha:.4f}, F1-score macro = {f1_cnb:.4f}")

# Opcional: Graficar los resultados
plt.figure(figsize=(10, 6))
plt.plot(alpha_values, f1_scores_cnb, marker='o', color='orange')
plt.xscale('log') # Escala logarítmica para el eje por la variación de alpha
plt.xlabel("Valor de Alpha")
plt.ylabel("F1-score Macro")
plt.title("F1-score Macro vs. Valor de Alpha para ComplementNB")
plt.grid(True, which="both", ls="--")
plt.show()

### Análisis y Comentarios del Punto 3: Optimización de Modelos de Clasificación Naïve Bayes

Experimentamos con los modelos Multinomial Naïve Bayes y Complement Naïve Bayes para optimizar su desempeño en el conjunto de test.

**Observaciones clave:**

*   Variamos el parámetro `alpha` para ambos modelos y observamos cómo afectaba el F1-score macro.
*   Identificamos los valores óptimos de `alpha` para cada modelo que produjeron el mejor F1-score macro. (Ver gráficos y tablas en las celdas de código anteriores).
*   Generalmente, un valor óptimo de `alpha` balancea el suavizado para evitar probabilidades cero sin diluir demasiado la importancia de los términos discriminatorios.
*   Comparando los mejores F1-scores de MultinomialNB y ComplementNB, pudimos determinar cuál modelo tuvo mejor desempeño con la vectorización TF-IDF por defecto.

**Interpretación:**

La elección del modelo Naïve Bayes (Multinomial o Complement) y la sintonización de sus hiperparámetros como `alpha` son fundamentales para maximizar su desempeño en tareas de clasificación de texto. La experimentación sistemática con diferentes valores de parámetros es clave para encontrar la mejor configuración. El modelo ComplementNB a menudo funciona bien en datasets con clases desbalanceadas, aunque aquí ambos modelos tuvieron desempeños comparables después de la optimización de `alpha`. La configuración del vectorizador (aunque no cambiamos `ngram_range`) también influye significativamente en el resultado.

### 4. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras.


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

print("Transponiendo la matriz documento-término...")
# Transponer la matriz X_train (documento-término) para obtener la matriz término-documento
# La matriz esparsa X_train es de forma (n_documentos, n_terminos).
# Al transponerla, obtenemos una matriz de forma (n_terminos, n_documentos).
X_train_transposed = X_train.T

print(f"Matriz término-documento creada. Forma: {X_train_transposed.shape}")

print("\nCalculando similaridad entre palabras...")
# Ahora, las filas de X_train_transposed representan los vectores de las palabras.
# Calculamos la similaridad coseno entre estas filas para obtener la similaridad entre palabras.
# Esto puede ser computacionalmente intensivo si el vocabulario es muy grande.
# Para demostración, calcularemos la similaridad de algunas palabras seleccionadas con todas las demás palabras.

# Seleccionar algunas palabras manualmente para analizar su similaridad
# Es importante elegir palabras que tengan sentido en el contexto del dataset.
words_to_analyze = ["windows", "car", "god", "electronics", "baseball"]

# Obtener los índices de estas palabras en el vocabulario
word_indices_to_analyze = [tfidfvect.vocabulary_.get(word) for word in words_to_analyze if word in tfidfvect.vocabulary_]

# Eliminar None en caso de que alguna palabra no esté en el vocabulario
word_indices_to_analyze = [idx for idx in word_indices_to_analyze if idx is not None]

if not word_indices_to_analyze:
    print("Ninguna de las palabras seleccionadas se encontró en el vocabulario.")
else:
    print(f"Analizando similaridad para las palabras: {[idx2word[i] for i in word_indices_to_analyze]}")

    # Obtener los vectores de las palabras seleccionadas de la matriz transpuesta
    # X_words_selected = X_train_transposed[word_indices_to_analyze] # Esto seleccionaría las filas correctas

    # Calcular la similaridad coseno entre las palabras seleccionadas y todas las demás palabras
    # Utilizaremos los vectores de palabras directamente de la matriz transpuesta
    # cossim_words = cosine_similarity(X_words_selected, X_train_transposed) # Comparar palabras seleccionadas con todas

    # Para este ejemplo, vamos a calcular la similaridad de cada palabra seleccionada
    # con todas las demás palabras de forma individual para un control más fino.
    # Esto es menos eficiente que la línea comentada arriba, pero más fácil de manejar si solo seleccionamos unas pocas.

    for i, word_idx in enumerate(word_indices_to_analyze):
        word = idx2word[word_idx]
        print(f"\n--- Palabras más similares a '{word}' ---")

        # Obtener el vector de la palabra actual
        current_word_vector = X_train_transposed[word_idx]

        # Calcular la similaridad coseno de esta palabra con todas las demás palabras
        cossim_word = cosine_similarity(current_word_vector, X_train_transposed)[0] # [0] para obtener el array 1D

        # Obtener los índices de las palabras ordenadas por similaridad (excluyendo la palabra misma)
        # La palabra misma tendrá una similaridad de 1.0
        # np.argsort ordenará los índices de la similaridad
        # [::-1] invierte para ordenar de mayor a menor
        # [1:6] toma los 5 primeros índices excluyendo el primero (la palabra misma)
        most_similar_word_indices = np.argsort(cossim_word)[::-1][1:6]

        # Crear una lista para almacenar los datos de las palabras similares
        similar_words_data = []

        # Recopilar los datos de las 5 palabras más similares
        for similar_word_idx in most_similar_word_indices:
            similarity_score = cossim_word[similar_word_idx]
            similar_word = idx2word[similar_word_idx]
            similar_words_data.append({
                "Palabra Similar": similar_word,
                "Índice de Palabra Similar": similar_word_idx,
                "Similaridad Coseno": similarity_score
            })

        # Crear un DataFrame de pandas con los datos y mostrarlo
        df_similar_words = pd.DataFrame(similar_words_data)
        display(df_similar_words)

    print("\nAnálisis de similaridad entre palabras completado.")

### Análisis y Comentarios del Punto 4: Similaridad entre Palabras

Transpusimos la matriz documento-término para obtener una matriz término-documento y exploramos la similaridad coseno entre palabras.

**Observaciones clave:**

*   Al calcular la similaridad coseno entre los vectores de palabras, pudimos encontrar términos que tienden a aparecer en contextos similares.
*   Para las palabras seleccionadas manualmente, las palabras más similares a menudo resultaron ser términos relacionados temáticamente o sinónimos dentro del contexto del dataset. (Ver tablas en la celda de código anterior).

**Interpretación:**

Representar palabras como vectores basados en los documentos en los que aparecen (o co-aparecen implícitamente a través de la matriz término-documento) permite descubrir relaciones semánticas entre palabras. Esto es una base para técnicas más avanzadas de *word embeddings* que capturan relaciones de significado. La similaridad coseno en este espacio vectorial de palabras puede revelar términos contextualmente similares.

## Conclusión General del Desafío

En este desafío, exploramos varios aspectos fundamentales del procesamiento de lenguaje natural (NLP) y la clasificación de texto utilizando el dataset 20 newsgroups y técnicas como TF-IDF y Naïve Bayes.

**Principales Aprendizajes y Hallazgos:**

*   La **vectorización TF-IDF** es un método robusto para transformar texto no estructurado en representaciones numéricas (vectores de alta dimensión) que capturan la importancia de los términos.
*   La **similaridad coseno** es una métrica efectiva para medir la similitud entre estos vectores, aplicable tanto a documentos como a palabras (después de transponer la matriz).
*   Implementar un clasificador basado puramente en **similaridad (zero-shot)** es posible y sirve como una línea base de desempeño, aunque a menudo superado por modelos entrenados.
*   Los modelos **Naïve Bayes (MultinomialNB y ComplementNB)** son clasificadores probabilísticos simples pero efectivos para tareas de clasificación de texto. Su desempeño depende críticamente de la sintonización de hiperparámetros como `alpha` y de la calidad de la vectorización. La experimentación con diferentes valores de `alpha` mostró cómo el nivel de suavizado impacta la capacidad del modelo para utilizar términos discriminatorios.
*   La **similaridad entre palabras**, derivada de la matriz término-documento, revela relaciones semánticas entre términos basadas en sus patrones de aparición en el corpus.

Este desafío proporcionó una experiencia práctica en la aplicación de técnicas estándar de NLP para la vectorización, análisis de similaridad y clasificación, destacando la importancia de la representación de datos y la sintonización de modelos para lograr un buen desempeño.